In [25]:
# Code:
# https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/acgan/acgan.py

In [26]:
# Imports
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [27]:
# Configs

cuda = False

n_epochs=200
batch_size=64
lr=0.0002
b1=0.5
b2=0.999
latent_dim=100
n_classes=10
img_size=32
channels=1
sample_interval=20


In [28]:
# Functions and Model
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


In [29]:
# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [30]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [31]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "../images/%d.png" % batches_done, nrow=n_row, normalize=True)
    

In [ ]:
# Training
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)


[Epoch 0/200] [Batch 0/938] [D loss: 1.498148, acc: 9%] [G loss: 1.508554]
[Epoch 0/200] [Batch 1/938] [D loss: 1.497787, acc: 11%] [G loss: 1.507713]
[Epoch 0/200] [Batch 2/938] [D loss: 1.497904, acc: 8%] [G loss: 1.507665]
[Epoch 0/200] [Batch 3/938] [D loss: 1.497959, acc: 9%] [G loss: 1.507322]
[Epoch 0/200] [Batch 4/938] [D loss: 1.497561, acc: 8%] [G loss: 1.506833]
[Epoch 0/200] [Batch 5/938] [D loss: 1.497791, acc: 9%] [G loss: 1.506875]
[Epoch 0/200] [Batch 6/938] [D loss: 1.497904, acc: 8%] [G loss: 1.506737]
[Epoch 0/200] [Batch 7/938] [D loss: 1.497782, acc: 12%] [G loss: 1.506469]
[Epoch 0/200] [Batch 8/938] [D loss: 1.497883, acc: 7%] [G loss: 1.506177]
[Epoch 0/200] [Batch 9/938] [D loss: 1.497540, acc: 16%] [G loss: 1.505599]
[Epoch 0/200] [Batch 10/938] [D loss: 1.497109, acc: 15%] [G loss: 1.504723]
[Epoch 0/200] [Batch 11/938] [D loss: 1.497531, acc: 13%] [G loss: 1.504702]
[Epoch 0/200] [Batch 12/938] [D loss: 1.497193, acc: 7%] [G loss: 1.504767]
[Epoch 0/200] [Ba

[Epoch 0/200] [Batch 107/938] [D loss: 1.495400, acc: 9%] [G loss: 1.526870]
[Epoch 0/200] [Batch 108/938] [D loss: 1.497007, acc: 15%] [G loss: 1.523069]
[Epoch 0/200] [Batch 109/938] [D loss: 1.497737, acc: 14%] [G loss: 1.524390]
[Epoch 0/200] [Batch 110/938] [D loss: 1.497457, acc: 13%] [G loss: 1.518049]
[Epoch 0/200] [Batch 111/938] [D loss: 1.497514, acc: 14%] [G loss: 1.518646]
[Epoch 0/200] [Batch 112/938] [D loss: 1.499322, acc: 16%] [G loss: 1.514588]
[Epoch 0/200] [Batch 113/938] [D loss: 1.498451, acc: 17%] [G loss: 1.510117]
[Epoch 0/200] [Batch 114/938] [D loss: 1.498335, acc: 12%] [G loss: 1.504641]
[Epoch 0/200] [Batch 115/938] [D loss: 1.496128, acc: 10%] [G loss: 1.506671]
[Epoch 0/200] [Batch 116/938] [D loss: 1.495417, acc: 17%] [G loss: 1.501188]
[Epoch 0/200] [Batch 117/938] [D loss: 1.495697, acc: 16%] [G loss: 1.499801]
[Epoch 0/200] [Batch 118/938] [D loss: 1.492818, acc: 17%] [G loss: 1.496751]
[Epoch 0/200] [Batch 119/938] [D loss: 1.491364, acc: 13%] [G los

[Epoch 0/200] [Batch 213/938] [D loss: 1.495741, acc: 20%] [G loss: 1.490902]
[Epoch 0/200] [Batch 214/938] [D loss: 1.490810, acc: 16%] [G loss: 1.489759]
[Epoch 0/200] [Batch 215/938] [D loss: 1.494229, acc: 26%] [G loss: 1.493207]
[Epoch 0/200] [Batch 216/938] [D loss: 1.487662, acc: 25%] [G loss: 1.490037]
[Epoch 0/200] [Batch 217/938] [D loss: 1.492506, acc: 23%] [G loss: 1.492589]
[Epoch 0/200] [Batch 218/938] [D loss: 1.494677, acc: 23%] [G loss: 1.490757]
[Epoch 0/200] [Batch 219/938] [D loss: 1.489209, acc: 24%] [G loss: 1.496707]
[Epoch 0/200] [Batch 220/938] [D loss: 1.494195, acc: 23%] [G loss: 1.494116]
[Epoch 0/200] [Batch 221/938] [D loss: 1.492618, acc: 19%] [G loss: 1.499205]
[Epoch 0/200] [Batch 222/938] [D loss: 1.494690, acc: 27%] [G loss: 1.494944]
[Epoch 0/200] [Batch 223/938] [D loss: 1.496628, acc: 21%] [G loss: 1.497934]
[Epoch 0/200] [Batch 224/938] [D loss: 1.499570, acc: 21%] [G loss: 1.498233]
[Epoch 0/200] [Batch 225/938] [D loss: 1.502715, acc: 24%] [G lo

[Epoch 0/200] [Batch 319/938] [D loss: 1.391784, acc: 42%] [G loss: 1.468189]
[Epoch 0/200] [Batch 320/938] [D loss: 1.416155, acc: 34%] [G loss: 1.483149]
[Epoch 0/200] [Batch 321/938] [D loss: 1.433609, acc: 32%] [G loss: 1.524684]
[Epoch 0/200] [Batch 322/938] [D loss: 1.399724, acc: 35%] [G loss: 1.506818]
[Epoch 0/200] [Batch 323/938] [D loss: 1.399046, acc: 35%] [G loss: 1.492302]
[Epoch 0/200] [Batch 324/938] [D loss: 1.410566, acc: 31%] [G loss: 1.509084]
[Epoch 0/200] [Batch 325/938] [D loss: 1.364809, acc: 39%] [G loss: 1.462684]
[Epoch 0/200] [Batch 326/938] [D loss: 1.394494, acc: 36%] [G loss: 1.453243]
[Epoch 0/200] [Batch 327/938] [D loss: 1.402651, acc: 36%] [G loss: 1.485531]
[Epoch 0/200] [Batch 328/938] [D loss: 1.374701, acc: 40%] [G loss: 1.493008]
[Epoch 0/200] [Batch 329/938] [D loss: 1.426009, acc: 31%] [G loss: 1.517607]
[Epoch 0/200] [Batch 330/938] [D loss: 1.394455, acc: 39%] [G loss: 1.458793]
[Epoch 0/200] [Batch 331/938] [D loss: 1.376678, acc: 40%] [G lo

[Epoch 0/200] [Batch 425/938] [D loss: 1.296468, acc: 56%] [G loss: 1.461508]
